<a href="https://colab.research.google.com/github/ValentinoVizner/google_Colab/blob/master/Kaggle_Grocery_Store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Brick-and-mortar grocery stores are always in a delicate dance with purchasing and sales forecasting. Predict a little over, and grocers are stuck with overstocked, perishable goods. Guess a little under, and popular items quickly sell out, leaving money on the table and customers fuming.

The problem becomes more complex as retailers add new locations with unique needs, new products, ever transitioning seasonal tastes, and unpredictable product marketing. Corporación Favorita, a large Ecuadorian-based grocery retailer, knows this all too well. They operate hundreds of supermarkets, with over 200,000 different products on their shelves.

Corporación Favorita has challenged the Kaggle community to build a model that more accurately forecasts product sales. They currently rely on subjective forecasting methods with very little data to back them up and very little automation to execute plans. They’re excited to see how machine learning could better ensure they please customers by having just enough of the right products at the right time.

In [1]:
!pip install fastai==0.7.0
!pip install "torchtext==0.2.3"
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v

     |████████████████████████████████| 122kB 9.6MB/s 
     |████████████████████████████████| 1.5MB 37.8MB/s 
     |████████████████████████████████| 496.4MB 31kB/s 
  Created wheel for bcolz: filename=bcolz-1.2.1-cp36-cp36m-linux_x86_64.whl size=2660625 sha256=ff4e81591fc3feacbb9db318336f1a8cc7c78dd8db207e8f78d65cb99fb797bf
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 0.3.1 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1
  Found existing installation: fastai 1.0.59
    Uninstalling fastai-1.0.59:
      Successfully uninstalled fastai-1.0.59
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [15]:
import os
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
os.environ['valentinovizner'] = "5ade780eb4c952eebf38c42bd1eab8ae" # username from the json file os.environ['KAGGLEKEY'] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx" # key from the json file

Saving kaggle.json to kaggle (1).json
  0% 0.00/33.9k [00:00<?, ?B/s]
100% 33.9k/33.9k [00:00<00:00, 24.1MB/s]


In [18]:
!kaggle competitions list -s favorita-grocery-sales-forecasting
!kaggle competitions download -c favorita-grocery-sales-forecasting -p /content/data/grocery_store


ref                                 deadline             category   reward  teamCount  userHasEntered  
----------------------------------  -------------------  --------  -------  ---------  --------------  
favorita-grocery-sales-forecasting  2018-01-15 23:59:00  Featured  $30,000       1674            True  
 99% 454M/458M [00:14<00:00, 27.3MB/s]
100% 458M/458M [00:14<00:00, 32.8MB/s]


In [34]:
#!unzip /content/data/grocery_storea/*.zip -d /content/data/grocery_storea/parent_unzip
!unzip /content/data/grocery_storea/parent_unzip/*.7z -d /content/data/grocery_storea/parent_unzip/unzipped

Archive:  /content/data/grocery_storea/parent_unzip/holidays_events.csv.7z
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/data/grocery_storea/parent_unzip/holidays_events.csv.7z or
        /content/data/grocery_storea/parent_unzip/holidays_events.csv.7z.zip, and cannot find /content/data/grocery_storea/parent_unzip/holidays_events.csv.7z.ZIP, period.


In [40]:
!7za x -y "/content/data/grocery_storea/parent_unzip/*.7z" -o/content/data/grocery_storea/parent_unzip/unzipped


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/data/grocery_storea/parent_unzip/                                                    8 files, 479884308 bytes (458 MiB)
  0%    
Extracting archive: /content/data/grocery_storea/parent_unzip/holidays_events.csv.7z
--
Path = /content/data/grocery_storea/parent_unzip/holidays_events.csv.7z
Type = 7z
Physical Size = 1898
Headers Size = 146
Method = LZMA2:24k
Solid = -
Blocks = 1

  0%    Everything is Ok

Extracting archive: /content/data/grocery_storea/parent_unzip/items.csv.7z
--
Path = /content/data/grocery_storea/parent_unzip/items.csv.7z
Type = 7z
Physical Size = 14315
Headers Size = 122
Method = LZMA2:17
Solid = -
B

In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [0]:
from fastai.imports import *
from fastai.structured import *


from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [0]:
PATH = "data/grocery_storea/"

In [23]:
!ls {PATH}

favorita-grocery-sales-forecasting.zip
